In [1]:
import pandas as pd
import numpy as np
import os
from keras.layers import Input, Dense
from keras.models import Model
import warnings
warnings.filterwarnings('ignore')

Using TensorFlow backend.


In [2]:
file_path="/home/externalFolder/demo_final_item/all_data_1919/combine_all_chr/all_chr_combine_data.csv"

In [3]:
#生成一个迭代器，用于训练

In [4]:
def generate_arrays_from_file(path,batch_size):
    while True:
        data = open(path)
        cnt = 0
        X =[]
        for line in data:
            x = np.array(line.strip().split(),dtype=np.float16)
            X.append(x)
            cnt += 1
            if cnt==batch_size:
                cnt = 0
                yield (np.array(X), np.array(X)) #返回一个数组，数组里面两个shape为(batch_size, 383800）的ndarry
                X = []
    f.close()

In [5]:
generator = generate_arrays_from_file(file_path,1000)

In [6]:
#读取验证数据集

In [7]:
valid_data_path="/home/externalFolder/demo_final_item/all_data_1919/combine_all_chr/valid_data.csv"

In [8]:
with open(valid_data_path) as valid_data_list:
    valid_data=[]
    for line in valid_data_list:
        line = np.array(line.strip().split(),dtype=np.float16)
        valid_data.append(line)
valid_data=np.array(valid_data)

In [9]:
#创建模型

In [10]:
encoding_dim = 100

In [11]:
input_img = Input(shape=(200*1919,))

In [12]:
encoded = Dense(encoding_dim, activation='relu')(input_img)

Instructions for updating:
Colocations handled automatically by placer.


In [13]:
decoded = Dense(200*1919, activation='relu')(encoded)

In [14]:
autoencoder = Model(input=input_img, output=decoded)

In [15]:
encoder = Model(input=input_img, output=encoded)

In [16]:
autoencoder.compile(optimizer='adam', loss='mse')

In [17]:
autoencoder.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 383800)            0         
_________________________________________________________________
dense_1 (Dense)              (None, 100)               38380100  
_________________________________________________________________
dense_2 (Dense)              (None, 383800)            38763800  
Total params: 77,143,900
Trainable params: 77,143,900
Non-trainable params: 0
_________________________________________________________________


In [18]:
encoder.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 383800)            0         
_________________________________________________________________
dense_1 (Dense)              (None, 100)               38380100  
Total params: 38,380,100
Trainable params: 38,380,100
Non-trainable params: 0
_________________________________________________________________


In [19]:
autoencoder.fit_generator(generator,
                    steps_per_epoch = 16,
                    epochs = 50,
                    verbose = 1,
                    validation_data = (valid_data,valid_data))

Instructions for updating:
Use tf.cast instead.


ValueError: Error when checking input: expected input_1 to have shape (383800,) but got array with shape (1,)

In [ ]:
loss_record = autoencoder.history.history['loss']
val_loss_record = autoencoder.history.history['val_loss']

with open("/home/externalFolder/demo_final_item/all_data_1919/loss","w") as loss:
    with open("/home/externalFolder/demo_final_item/all_data_1919/val_loss","w") as val_loss:
        loss.writelines(str(loss_record))
        val_loss.writelines(str(val_loss_record))

In [ ]:
#保存模型

In [ ]:
layer_name = 'dense_1'
intermediate_layer_model = Model(input=encoder.input,
                                 output=encoder.get_layer(layer_name).output)

In [ ]:
#数据预测

In [ ]:
def predictResult(chr_num):
    predict_data_path='/home/externalFolder/demo_final_item/all_data_1919/chr'+str(chr_num)+".txt"
    result_save_path="/home/externalFolder/demo_final_item/all_data_1919/result/chr"+str(chr_num)+".npy"
    with open(predict_data_path) as predict_data_list:
        predict_data=[]
        for line in predict_data_list:
            line = np.array(line.strip().split(),dtype=np.float16)
            predict_data.append(line)
    
    intermediate_output = intermediate_layer_model.predict(np.array(predict_data))
    np.save(result_save_path,intermediate_output)

In [ ]:
predictResult(1)

In [ ]:
predictResult(3)
predictResult(4)
predictResult(5)
predictResult(6)
predictResult(7)
predictResult(8)
predictResult(9)
predictResult(10)
predictResult(11)
predictResult(12)
predictResult(13)
predictResult(14)
predictResult(15)
predictResult(16)
predictResult(18)
predictResult(19)
predictResult(20)
predictResult(21)
predictResult(22)